In [24]:
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from google.colab import drive
drive.mount('/content/drive')
DRIVE_PATH = '/content/drive/MyDrive/multiclass/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
org = pd.read_csv(f'{DRIVE_PATH}/cirrhosis.csv')
org.dropna(axis=0, subset = ['Status'], inplace=True)
org.rename(columns={'ID': 'id'}, inplace=True)
df = pd.read_csv(f'{DRIVE_PATH}/train.csv')
df = pd.concat([df,org], axis=0)

df['Age'] = df['Age']/365
df['Age'] = np.ceil(df['Age'])


X_  = df.drop(['id','Status'],axis=1)
y_  = df['Status']


X_new = X_.replace(True,1).replace(False,0).replace('M',1).replace('F',0).replace('Y',1).replace('N',0).replace('S',1).replace('Placebo',1).replace('D-penicillamine',0)
y_new = LabelEncoder().fit_transform(y_)
Params = {'max_depth': 6,
          'learning_rate': 0.03711180731953549,
          'n_estimators': 390,
          'min_child_weight': 8,
          'gamma': 8.878825928653752e-05,
          'subsample': 0.5871365948757953,
          'colsample_bytree': 0.175999259700574,
          'reg_alpha': 1.0782222832975881e-08,
          'reg_lambda': 0.002131207411566484}

In [26]:
model = XGBClassifier(**Params)
model.fit(X_new, y_new)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.175999259700574, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=8.878825928653752e-05,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.03711180731953549,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=8, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=390, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [13]:
class_distribution_train = np.bincount(y_new) / len(y_new)
class_distribution_train

array([0.62441427, 0.0360447 , 0.33954103])

In [87]:
df_test = pd.read_csv(f'{DRIVE_PATH}/test.csv')
df_test['Age'] = df_test['Age']/365
df_test['Age'] = np.ceil(df_test['Age'])
sub = model.predict_proba(df_test.drop('id',axis=1).replace(True,1).replace(False,0).replace('M',1).replace('F',0).replace('Y',1).replace('N',0).replace('S',1).replace('Placebo',1).replace('D-penicillamine',0))
submission = pd.DataFrame(data=sub,
                          columns =['Status_C', 'Status_CL', 'Status_D'],
                          index=df_test['id'])
submission.reset_index().to_csv(f'{DRIVE_PATH}/xgb_alldata.csv',index=False)


In [8]:
submission.head()

,Status_C,Status_CL,Status_D
id,,,
7905,0.485088,0.035697,0.479215
7906,0.509031,0.220636,0.270333
7907,0.044129,0.008680,0.947191
7908,0.974874,0.003653,0.021472
7909,0.842567,0.065812,0.091621


In [9]:
submission.shape

(5271, 3)

In [88]:
sub_single = model.predict(df_test.drop('id',axis=1).replace(True,1).replace(False,0).replace('M',1).replace('F',0).replace('Y',1).replace('N',0).replace('S',1).replace('Placebo',1).replace('D-penicillamine',0))
class_distribution_test = np.bincount(sub_single) / len(sub_single)
np.bincount(sub_single)


array([3514,   48, 1709])

In [72]:
sub_single[:10]

array([0, 0, 2, 0, 0, 0, 0, 2, 2, 0])

In [11]:
class_distribution_test

array([0.66666667, 0.00910643, 0.3242269 ])

In [17]:
[5271*0.62441427, 5271*0.0360447 , 5271*0.33954103]
#3291, 190, 1790

[3291.2876171700004, 189.9916137, 1789.72076913]

In [20]:
3514-3291#92

223

In [21]:
190-48#50

142

In [56]:
1790-1709#42+26

81

In [111]:
df_test = pd.read_csv(f'{DRIVE_PATH}/test.csv')
df_test['Age'] = df_test['Age']/365
df_test['Age'] = np.ceil(df_test['Age'])
sub = model.predict_proba(df_test.drop('id',axis=1).replace(True,1).replace(False,0).replace('M',1).replace('F',0).replace('Y',1).replace('N',0).replace('S',1).replace('Placebo',1).replace('D-penicillamine',0))
submission = pd.DataFrame(data=sub,
                          columns =['Status_C', 'Status_CL', 'Status_D'],
                          index=df_test['id'])
submission = submission.sort_values('Status_CL',ascending=False)
i = 0
count = 0
dec_death = 0
for row in submission.iterrows():
  if(row[1]['Status_CL']<row[1]['Status_C']):
    submission.iloc[count]['Status_CL'],submission.iloc[count]['Status_C'] = submission.iloc[count]['Status_C'], submission.iloc[count]['Status_CL']
    i+=1
  elif (row[1]['Status_CL']<row[1]['Status_D']):
    submission.iloc[count]['Status_CL'],submission.iloc[count]['Status_D'] = submission.iloc[count]['Status_D'], submission.iloc[count]['Status_CL']
    i+=1
    dec_death+=1
  count+=1
  if(i==83):
    break

In [112]:
dec_death

26

In [113]:
submission = submission.sort_values('Status_D',ascending=False)
i = 0
count = 0
dec_death = 0
for row in submission.iterrows():
  if(row[1]['Status_D']<row[1]['Status_C']):
    submission.iloc[count]['Status_D'],submission.iloc[count]['Status_C'] = submission.iloc[count]['Status_C'], submission.iloc[count]['Status_D']
    i+=1
  count+=1
  if(i==68):
    break

In [114]:
rank_sub = submission.idxmax(axis = 1)
rank_sub.value_counts()/rank_sub.shape[0]
#Preprocess: [3514,   48, 1709]
#Target: [3291, 190, 1790]

Status_C     0.646177
Status_D     0.332195
Status_CL    0.021628
dtype: float64

In [109]:
rank_sub = submission.idxmax(axis = 1)
rank_sub.value_counts()/rank_sub.shape[0]
#array([0.62441427, 0.0360447 , 0.33954103])

Status_C     0.628913
Status_D     0.339973
Status_CL    0.031114
dtype: float64

In [73]:
submission.head()

,Status_C,Status_CL,Status_D
id,,,
9076,0.001674,0.000673,0.997653
8086,0.001596,0.000898,0.997505
11069,0.001809,0.000699,0.997492
12734,0.002174,0.000620,0.997206
11942,0.002218,0.000893,0.996888


In [115]:
submission.reset_index().to_csv(f'{DRIVE_PATH}/postproc.csv',index=False)

In [17]:
import matplotlib.pyplot as plt

y_pred = model.predict(X_new)
print(classification_report(y_new, y_pred))
print(log_loss(y_new, y_pred))
plt.plot(model.loss_)
plt.show()

              precision    recall  f1-score   support

           0       0.89      0.95      0.92      5197
           1       0.90      0.35      0.50       300
           2       0.88      0.82      0.85      2826

    accuracy                           0.89      8323
   macro avg       0.89      0.71      0.76      8323
weighted avg       0.89      0.89      0.88      8323



ValueError: ignored

In [18]:
# prompt: plot the error from model

plt.plot(model.loss_)
plt.show()


AttributeError: ignored

In [ ]:
# prompt: plot error, log los and accuracy from model

plt.plot(model.loss_)
plt.show()

plt.plot(model.log_loss_)
plt.show()

plt.plot(model.score_)
plt.show()
